In [166]:
import lyricsgenius
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
import bs4
from bs4 import BeautifulSoup
import re
import lxml.html
from lxml import html
from datetime import datetime, timedelta
import time
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
stop = stopwords.words('english')
sno = SnowballStemmer('english')


genius = lyricsgenius.Genius("lSUVIEFu57a_JUG5gUTMqPRgOUIBqNJJtSWWqaG1LnBc2FMi8qveGlBapwaTCRJe")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/efrancois/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [125]:


dom = lxml.html.fromstring(requests.get('https://genius.com/discussions/22873-Greatest-100-rappers').content)
rappers = [x for x in dom.xpath('//li/text()')]
rappers = [i for i in rappers if '\n' not in i]


base_url = "https://www.billboard.com/charts/artist-100/"
pop_big = []
for n in range(10):
    d = (datetime.now() - timedelta(days=60*n)).strftime('%Y-%m-%d')
    dom2 = lxml.html.fromstring(requests.get(base_url + d).content)
    pop = [x for x in dom2.xpath('//*[@class="chart-list-item__title-text"]/a/text()')]
    pop = [i[1:-1] for i in pop]
    pop_big += pop
    time.sleep(.1)
pop_big = list(set(pop_big))


dom3 = lxml.html.fromstring(requests.get('https://www.billboard.com/articles/news/dance/8504278/billboard-dance-100-2019').content)
electronic = [x for x in dom3.xpath('//p/strong/text()')]

no_words = ['Top Gig of 2018:', 'Outlook:', 'Notable Stat:','Contributors: ']
electronic=[i for i in electronic if not any(words in i for words in no_words)]


print(electronic)


['100. Bob Moses', '99. Lost Kings', '98. TOKiMONSTA', '97. Slushii', '96. Whethan', '95. Petit Biscuit', '94. Nina Kraviz', '93. CamelPhat', '92. Carnage', '91. Netsky', '90. Jax Jones', '89. The Martinez Brothers', '88. Felix Jaehn', '87. Showtek', '86. Lost Frequencies', '85. Deorro', '84. Marco Carola', '83. Jai Wolf', '82. NERVO', '81. Fisher', '80. Seven Lions', '79. MK', '78. Loud Luxury', '77. Nicky Romero', '76. Cash Cash', '75. Jonas Blue', '74. GRiZ', '73. Sofi\xa0Tukker', '72. 3LAU', '71. Alok', '70. SNAILS', '69. Madeon', '68. Maceo Plex', '67. Kayzo', '66. Big Gigantic', '65. KSHMR', '64. Richie Hawtin', '63. Oliver Heldens', '62. Adam Beyer', '61. ZHU', '60. Jauz', '59. Pretty Lights', '58. Matoma', '57. Black Coffee', '56. San Holo', '55. Gryffin', '54. Cashmere Cat', '53. Mura Masa', '52. W&W', '51. NGHTMRE', '50. Bonobo', '49. Robin Schulz', '48. Tchami', '47. R3hab', '46. Louis The Child', '45. Jamie Jones', '44. Cheat Codes', '43. RL Grime', '42. Yellow Claw', '41. 

In [126]:
electronic_fixed = []
for elec in electronic:
    electronic_fixed.append('. '.join(elec.split('. ')[1:]))
print(electronic_fixed)

['Bob Moses', 'Lost Kings', 'TOKiMONSTA', 'Slushii', 'Whethan', 'Petit Biscuit', 'Nina Kraviz', 'CamelPhat', 'Carnage', 'Netsky', 'Jax Jones', 'The Martinez Brothers', 'Felix Jaehn', 'Showtek', 'Lost Frequencies', 'Deorro', 'Marco Carola', 'Jai Wolf', 'NERVO', 'Fisher', 'Seven Lions', 'MK', 'Loud Luxury', 'Nicky Romero', 'Cash Cash', 'Jonas Blue', 'GRiZ', 'Sofi\xa0Tukker', '3LAU', 'Alok', 'SNAILS', 'Madeon', 'Maceo Plex', 'Kayzo', 'Big Gigantic', 'KSHMR', 'Richie Hawtin', 'Oliver Heldens', 'Adam Beyer', 'ZHU', 'Jauz', 'Pretty Lights', 'Matoma', 'Black Coffee', 'San Holo', 'Gryffin', 'Cashmere Cat', 'Mura Masa', 'W&W', 'NGHTMRE', 'Bonobo', 'Robin Schulz', 'Tchami', 'R3hab', 'Louis The Child', 'Jamie Jones', 'Cheat Codes', 'RL Grime', 'Yellow Claw', 'Solomun', 'Dillon Francis', 'Rezz', 'Don Diablo', 'Zeds Dead', 'Alison Wonderland', 'Illenium', 'Clean Bandit', 'Porter Robinson', 'Alan Walker', 'Afrojack', '', 'Disclosure', 'Carl Cox', 'Galantis', 'Eric Prydz', 'Kaskade', 'Justice', 'Bass

In [129]:
artists=rappers+pop_big+electronic_fixed

artists = list(set(artists))

print(len(artists))

468


In [130]:

artists = [art.replace('The Jungle Brothers', 'Jungle Brothers') for art in artists]
artists = [art.replace('The Funky Four+1', 'Funky Four Plus One') for art in artists]
artists = [art.replace('Kool Keith/Dr. Octagon', 'Kool Keith') for art in artists]

print(artists)
#print(len(artists))

# artists.append('Kool Keith')
#artists.index('Jungle Brothers')
#artists.index('Funky Four Plus One')
#artists.index('Kool Keith')



['', 'Salt N Pepa', 'Sam Hunt', ' Backstreet Boys', 'Lee Brice', 'Chance The Rapper', 'San Holo', 'Flume', 'Raekwon', 'Highlands Worship', 'Lil Pump', 'Quavo', ' Cardi B', 'Michael Buble', 'Amos Lee', 'Eve', 'Blake Shelton', 'Richie Hawtin', 'Tyler, The Creator', 'Stick Figure', 'Cashmere Cat', 'Lil Dicky', 'Alison Wonderland', 'Shinedown', 'Gang Starr', 'Black Coffee', 'Ringo Starr', 'Billy Ray Cyrus', 'Keith Urban', '5 Seconds Of Summer', 'Ice Cube', 'Eightball & MJG', 'Fisher', '50 Cent', 'BLACKPINK', 'Bruno Mars', 'George Strait', ' Bradley Cooper', 'Ty Dolla $ign', 'dead prez', 'Bone Thugs N Harmony', 'Alan Jackson', 'The Raconteurs', 'Stetsasonic', '2Pac', 'Big Daddy Kane', 'Jungle Brothers', 'Ras Kass', 'Ice-T', 'Tool', 'Lauren Daigle', 'Jason Aldean', 'Bebe Rexha', 'Jean Grae', 'Godsmack', 'Creedence Clearwater Revival', 'Kelly Clarkson', 'Big L', 'Foster The People', 'Morgan Wallen', 'Travis Scott', 'Marco Carola', 'for KING & COUNTRY', 'Kacey Musgraves', 'Dustin Lynch', 'Stev

In [135]:
lyric_dict = {}
artist_dict = {}
for art in artists:
    try:
        artist = genius.search_artist(art, max_songs=10,sort='popularity')
    except requests.exceptions.Timeout:
        continue
    if artist is None:
        continue
    for song in artist.songs:
        #print(song.artist + ' ' + song.title)
        lyric_dict[song.artist + '_' + song.title] = song.lyrics
        artist_dict[song.artist + '_' + song.title] = song.artist

    

Searching for songs by ...

No results found for ''.
Searching for songs by Salt N Pepa...

Changing artist name to 'Salt-N-Pepa'
Song 1: "Shoop"
Song 2: "Push It"
Song 3: "Let’s Talk About Sex"
Song 4: "Whatta Man"
Song 5: "None of Your Business"
Song 6: "I’ll Take Your Man"
Song 7: "Ain’t Nuthin’ But a She Thing"
Song 8: "Do You Want Me (Remix)"
Song 9: "I Am the Body Beautiful"
Song 10: "Tramp (Remix)"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Sam Hunt...

Song 1: "Body Like a Back Road"
Song 2: "Take Your Time"
Song 3: "Break Up in a Small Town"
Song 4: "Drinkin’ Too Much"
Song 5: "Make You Miss Me"
Song 6: "Downtown’s Dead"
Song 7: "Leave the Night On"
Song 8: "Speakers"
Song 9: "Ex to See"
Song 10: "Single for the Summer"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by  Backstreet Boys...

Changing artist name to 'Backstreet Boys'
Song 1: "I Want It That Way"
Song 2: "Don’t Go Breaking My Heart"
Son

Song 1: "Moment of Truth"
Song 2: "Above the Clouds"
Song 3: "Full Clip"
Song 4: "Work"
Song 5: "Family and Loyalty"
Song 6: "DWYCK"
Song 7: "Mass Appeal"
Song 8: "You Know My Steez"
Song 9: "Betrayal"
Song 10: "JFK 2 LAX"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Black Coffee...

Song 1: "Your Eyes"
Song 2: "Superman"
Song 3: "Wish You Were Here"
Song 4: "Turn Me On"
Song 5: "LaLaLa"
Song 6: "Superman (club mix)"
Song 7: "We Dance Again"
Song 8: "Come With Me"
Song 9: "Ndod’ emnyama"
Song 10: "Stimela"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Ringo Starr...

Song 1: "The No-No Song"
Song 2: "It Don’t Come Easy"
Song 3: "You’re Sixteen (You’re Beautiful You’re Mine)"
Song 4: "Photograph"
Song 5: "Give More Love"
Song 6: "Grow Old with Me"
Song 7: "We’re On the Road Again"
Song 8: "I’m The Greatest"
Song 9: "What’s My Name"
Song 10: "Back Off Boogaloo"

Reached user-specified song limit (10).
Done. 

Song 2: "Straight Out the Jungle"
Song 3: "Doin’ Our Own Dang"
Song 4: "Because I Got it Like That"
Song 5: "Jungle Brother (True Blue)"
Song 6: "Black Is Black"
Song 7: "What ‘U’ Waiting For?"
Song 8: "Jimbrowski"
Song 9: "I’ll House You"
Song 10: "I’ll House You (House mix)"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Ras Kass...

Song 1: "Nature of the Threat"
"Interview With a Vampire" is not valid. Skipping.
Song 2: "On Earth As It Is..."
Song 3: "Soul on Ice"
Song 4: "Goldyn Chyld"
Song 5: "Anything Goes"
Song 6: "Soul On Ice (Remix)"
Song 7: "Sushi"
Song 8: "The Evil That Men Do"
Song 9: "Oral Sex (’99)"
Song 10: "Miami Life"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Ice-T...

Song 1: "Colors"
Song 2: "6 ‘N the Mornin’"
Song 3: "99 Problems"
Song 4: "I’m Your Pusher"
Song 5: "Girls L.G.B.N.A.F."
Song 6: "New Jack Hustler"
Song 7: "O.G. Original Gangster"
Song 8: "Body Count"
Song 9: "Warning"
S

Song 9: "Turn It Up"
Song 10: "Heading Up High"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Alessia Cara...

Song 1: "Here"
Song 2: "How Far I’ll Go"
Song 3: "Scars to Your Beautiful"
Song 4: "Wild Things"
Song 5: "I’m Yours"
Song 6: "Seventeen"
Song 7: "River of Tears"
Song 8: "Growing Pains"
Song 9: "Out of Love"
Song 10: "Outlaws"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Jonas Blue...

Song 1: "Perfect Strangers"
Song 2: "Mama"
Song 3: "Rise"
Song 4: "Fast Car"
Song 5: "Polaroid"
Song 6: "By Your Side"
Song 7: "We Could Go Back"
Song 8: "I See Love"
Song 9: "What I Like About You"
Song 10: "Wild"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Oliver Heldens...

Changing artist name to 'Oliver Heldens & Becky Hill'
Song 1: "Gecko (Overdrive)"
Done. Found 1 songs.
Searching for songs by Halsey...

Song 1: "Without Me"
Song 2: "Colors"
Song 3: "Bad at Love"
Song

Song 9: "When the Sun Goes Down"
Song 10: "The Good Stuff"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Bobby "Boris" Pickett And The Crypt-Kickers...

No results found for 'Bobby "Boris" Pickett And The Crypt-Kickers'.
Searching for songs by Jon Pardi...

Song 1: "Night Shift"
Song 2: "Dirt On My Boots"
Song 3: "Head Over Boots"
Song 4: "Heartache On The Dance Floor"
Song 5: "She Ain’t In It"
Song 6: "Heartache Medication"
Song 7: "Can’t Turn You Down"
Song 8: "Over My Head"
Song 9: "When I’ve Been Drinkin’"
Song 10: "Nobody Leaves a Girl Like That"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Pistol Annies...

Song 1: "Best Years of My Life"
Song 2: "Got My Name Changed Back"
Song 3: "Interstate Gospel"
Song 4: "When I Was His Wife"
Song 5: "Sugar Daddy"
Song 6: "Stop Drop and Roll One"
Song 7: "Cheyenne"
Song 8: "Hell On Heels"
Song 9: "Leavers Lullaby"
Song 10: "I Feel a Sin Comin’ On"

Reached user-s

...

Song 8: "Lyin’ Eyes"
Song 9: "Take It to the Limit"
Song 10: "Wasted Time"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by NF...

Song 1: "Let You Down"
Song 2: "The Search"
Song 3: "Lie"
Song 4: "WHY"
Song 5: "When I Grow Up"
Song 6: "How Could You Leave Us"
Song 7: "Remember This"
Song 8: "If You Want Love"
Song 9: "Mansion"
Song 10: "Leave Me Alone"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Dr. Dre...

Changing artist name to 'Dr. Dre'
Song 1: "Forgot About Dre"
Song 2: "Still D.R.E."
Song 3: "Nuthin’ But a ‘G’ Thang"
Song 4: "The Next Episode"
Song 5: "I Need a Doctor"
Song 6: "What’s the Difference"
Song 7: "Medicine Man"
Song 8: "Bitches Ain’t Shit"
Song 9: "Fuck wit Dre Day (And Everybody’s Celebratin’)"
Song 10: "Deep Water"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Guns N' Roses...

Changing artist name to 'Guns N’ Roses'
Song 1: "Sweet Child O’ Mine"
S

Song 3: "Saturnz Barz"
Song 4: "Ascension"
Song 5: "Andromeda"
Song 6: "On Melancholy Hill"
Song 7: "Humility"
Song 8: "DARE"
Song 9: "November Has Come"
Song 10: "Let Me Out"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Weezer...

Song 1: "Say It Ain’t So"
Song 2: "Buddy Holly"
Song 3: "Island in the Sun"
Song 4: "Undone - The Sweater Song"
Song 5: "El Scorcho"
Song 6: "Thank God for Girls"
Song 7: "Hash Pipe"
Song 8: "Pink Triangle"
Song 9: "My Name Is Jonas"
Song 10: "Beverly Hills"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Grateful Dead...

Song 1: "Terrapin Station"
Song 2: "Ripple"
Song 3: "Scarlet Begonias"
Song 4: "Truckin’"
Song 5: "Friend of the Devil"
Song 6: "Box of Rain"
Song 7: "Touch of Grey"
Song 8: "Casey Jones"
Song 9: "Uncle John’s Band"
Song 10: "Sugar Magnolia"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by  Brendon Urie...

Changing artist na

Song 3: "Star Crossed"
Song 4: "How You Love Me"
Song 5: "On My Own"
Song 6: "On My Mind"
Song 7: "You Want More"
Song 8: "Close"
Song 9: "Walk Away"
Song 10: "Worlds Away"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by AC/DC...

Song 1: "Back in Black"
Song 2: "Thunderstruck"
Song 3: "Highway to Hell"
Song 4: "You Shook Me All Night Long"
Song 5: "T.N.T."
Song 6: "Dirty Deeds Done Dirt Cheap"
Song 7: "Big Balls"
Song 8: "Hells Bells"
Song 9: "Shoot to Thrill"
Song 10: "It’s a Long Way to the Top (If You Wanna Rock n’ Roll)"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Demon Hunter...

Song 1: "Jesus Wept"
Song 2: "On My Side"
Song 3: "Dead Flowers"
Song 4: "Died in My Sleep"
Song 5: "I Will Fail You"
Song 6: "I Am a Stone"
Song 7: "Death"
Song 8: "God Forsaken"
Song 9: "Someone to Hate"
Song 10: "Cold Winter Sun"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Illenium

Song 6: "HIT"
Song 7: "Burning - vibe mix"
Song 8: "Always"
Song 9: "Always (Route 94 Remix)"
Song 10: "JDZmedia - MK Spitfire PT.2"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Chris Janson...

Song 1: "Drunk Girl"
Song 2: "Buy Me a Boat"
Song 3: "Holdin’ Her"
Song 4: "Fix a Drink"
Song 5: "Everybody"
Song 6: "Good Vibes"
Song 7: "Bein’ a Dad"
Song 8: "Little Bit of Both"
Song 9: "Who’s Your Farmer"
Song 10: "Power of Positive Drinkin’"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Chase Rice...

Song 1: "Ride"
Song 2: "If I’m Bein’ Honest"
Song 3: "Eyes on You"
Song 4: "Three Chords & the Truth"
Song 5: "Lions"
Song 6: "Saved Me"
Song 7: "On Tonight"
Song 8: "One Love, One Kiss, One Drink, One Song"
Song 9: "Gonna Wanna Tonight"
Song 10: "Unforgettable"

Reached user-specified song limit (10).
Done. Found 10 songs.
Searching for songs by Petit Biscuit...

Song 1: "Sunset Lover"
Song 2: "Problems"
Song 3:

TypeError: argument of type 'NoneType' is not iterable

In [170]:
lyric_df['lyrics']

0       [Intro Pepa]\n(Hey yeah, I wanna shoop baby!)\...
1       Ah, push it\nAh, push it\n\n(Hit it)\n\nOooh, ...
2       [Intro: Punch it, (Hurb)]\nYo, I don't think w...
3       [Intro]\nYeee-hey, yeah\nAlright\nCome on now\...
4       [Intro: Salt n Pepa]\nWhat's the matter with y...
5       Salt and Pepa's back, and we came to out rap y...
6       It ain't a man's world (You go, girl!)\nNo mor...
7       Artist: Salt N Pepa f/ Alpha Omega\nAlbum:  Gr...
8       [Intro]\nAnd where is the body?\nAw yeah, don'...
9       What'd ya call me?\n\n[Yo, yo, D, yo, check ou...
10      [Verse 1]\nGot a girl from the south side, got...
11      [Verse 1]\nI don't know if you were looking at...
12      [Spoken Verse 1]\nI knew I'd see her around\nI...
13      [Intro]\nPour a drink and take a sip of it\nFe...
14      [Verse 1]\nYou hear a new song, it's your favo...
15      [Intro]\n"Thanks, Hannah, thanks for coming ba...
16      [Verse 1]\nThey roll the sidewalks in this tow...
17      [Verse

In [167]:
def clean_html(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

def clean_punc(word):
    cleaned = re.sub(r'[?|!|\'|#]', r'', word)
    cleaned = re.sub(r'[.|,|)|(|\|/]', r' ', cleaned)
    return cleaned


final_string = []
s = ''
for sentence in lyric_df['lyrics'].values:
    filtered_sentence = []
    sentence = clean_html(sentence)
    for word in sentence.split():
        for cleaned_word in clean_punc(word).split():
            if (cleaned_word.isalpha() and (len(cleaned_word) > 2) and cleaned_word not in stop):
                s = (sno.stem(cleaned_word.lower())).encode('utf8')
                filtered_sentence.append(s)
            else:
                continue
                 
    strl = b' '.join(filtered_sentence)
    final_string.append(strl)

lyric_df['Cleaned Text'] = final_string

In [169]:
lyric_df['Cleaned Text']

0       b'hey yeah wanna shoop babi how babi one yeah ...
1       b'push push hit oooh babi babi babi babi oooh ...
2       b'punch hurb dont think talk come peopl might ...
3       b'yeah alright come yeah talk yeah whatta man ...
4       b'salt what matter life whi gotta mess mine do...
5       b'salt pepa back came rap get face smack dont ...
6       b'aint man world you girl sugar spice and ever...
7       b'salt pepa alpha omega greatest hit you want ...
8       b'and bodi yeah dont hate caus beauti and bodi...
9       b'whatd call check girl man readi let roll you...
10      b'got girl south side got braid hair first tim...
11      b'dont know look you probabl smile like time a...
12      b'vers knew see around parti shed show walk ac...
13      b'pour drink take sip feelin like hypocrit cou...
14      b'you hear new song favorit but pretti soon yo...
15      b'hannah thank come the style all see mile peo...
16      b'they roll sidewalk town sun goe they say not...
17      b'soft

In [173]:
lyric_df_clean = pd.DataFrame.from_dict(lyric_dict,orient='index').reset_index()
lyric_df_clean.columns = ['song','lyrics']
cv = CountVectorizer()
df = cv.fit_transform(lyric_df_clean['Cleaned Text'])

count_vect_df_clean = pd.DataFrame(df.todense(), columns=['vec_' + i for i in cv.get_feature_names()])
lyric_df_clean = lyric_df_clean.join(count_vect_df_clean)

print(lyric_df_clean)

KeyError: 'Cleaned Text'

In [136]:
lyric_df = pd.DataFrame.from_dict(lyric_dict,orient='index').reset_index()
lyric_df.columns = ['song','lyrics']
cv = CountVectorizer()
df = cv.fit_transform(lyric_df['lyrics'])

count_vect_df = pd.DataFrame(df.todense(), columns=['vec_' + i for i in cv.get_feature_names()])
lyric_df = lyric_df.join(count_vect_df)

print(lyric_df)
#print(count_vect_df)

                                                   song  \
0                                     Salt-N-Pepa_Shoop   
1                                   Salt-N-Pepa_Push It   
2                      Salt-N-Pepa_Let’s Talk About Sex   
3                                Salt-N-Pepa_Whatta Man   
4                     Salt-N-Pepa_None of Your Business   
5                        Salt-N-Pepa_I’ll Take Your Man   
6             Salt-N-Pepa_Ain’t Nuthin’ But a She Thing   
7                    Salt-N-Pepa_Do You Want Me (Remix)   
8                   Salt-N-Pepa_I Am the Body Beautiful   
9                             Salt-N-Pepa_Tramp (Remix)   
10                       Sam Hunt_Body Like a Back Road   
11                              Sam Hunt_Take Your Time   
12                    Sam Hunt_Break Up in a Small Town   
13                           Sam Hunt_Drinkin’ Too Much   
14                            Sam Hunt_Make You Miss Me   
15                             Sam Hunt_Downtown’s Dead 

In [176]:
artist_df = pd.DataFrame.from_dict(artist_dict,orient='index').reset_index()
artist_df.columns = ['song','artist']
music_df = artist_df.merge(lyric_df,on='song')

#print(artist_df)
#print(music_df)

In [177]:
music_df.dtypes

song            object
artist          object
lyrics          object
vec_00           int64
vec_000          int64
vec_01           int64
vec_02           int64
vec_03           int64
vec_04           int64
vec_05           int64
vec_06           int64
vec_08           int64
vec_09           int64
vec_10           int64
vec_100          int64
vec_1000         int64
vec_100k         int64
vec_101          int64
vec_103          int64
vec_105          int64
vec_106          int64
vec_108          int64
vec_10k          int64
vec_10s          int64
vec_10th         int64
vec_11           int64
vec_110          int64
vec_112          int64
vec_116th        int64
vec_11s          int64
                 ...  
vec_혹해           int64
vec_혼란스러         int64
vec_홀로           int64
vec_화나면          int64
vec_화만           int64
vec_효도           int64
vec_후회           int64
vec_후회하는         int64
vec_후회하지         int64
vec_훔쳐           int64
vec_훔쳐보다가        int64
vec_흉터들로         int64
vec_흐르게    

In [138]:
averaged_df = music_df.drop(['song','lyrics'],axis=1).groupby('artist').mean().reset_index()



                                        artist  vec_00  vec_000  vec_01  \
0                                     2 Chainz     0.0      0.0     0.0   
1                                    21 Savage     0.0      0.0     0.0   
2                     21 Savage & Metro Boomin     0.0      0.0     0.0   
3                                         2Pac     0.0      0.0     0.0   
4                                         3LAU     0.0      0.0     0.0   
5                          5 Seconds of Summer     0.0      0.0     0.0   
6                                      50 Cent     0.0      0.0     0.0   
7                                  8Ball & MJG     0.0      0.0     0.0   
8                       A Boogie Wit da Hoodie     0.0      0.0     0.0   
9                                        AC/DC     0.0      0.0     0.0   
10                              Above & Beyond     0.0      0.0     0.0   
11                              Action Bronson     0.0      0.1     0.0   
12                     Ad

In [141]:
from sklearn.metrics.pairwise import cosine_similarity
query='JAY-Z'
match = averaged_df[averaged_df['artist']==query]
rest_of_entries = averaged_df[averaged_df['artist']!=query].copy()

if match.empty:
    pass
    #Throw something here
    #return None
print(cosine_similarity(match.loc[:, match.columns != 'artist'],rest_of_entries.loc[:,rest_of_entries.columns!='artist']))
rest_of_entries['similarity'] = cosine_similarity(match.loc[:, match.columns != 'artist'],rest_of_entries.loc[:,rest_of_entries.columns!='artist'])[0]
rest_of_entries

ValueError: Found array with 0 sample(s) (shape=(0, 33882)) while a minimum of 1 is required by check_pairwise_arrays.

In [140]:
cosine_similarity(averaged_df.drop('artist',axis=1),averaged_df.drop('artist',axis=1))

array([[1.        , 0.82064323, 0.70700936, ..., 0.74918614, 0.69364512,
        0.6932935 ],
       [0.82064323, 1.        , 0.76728803, ..., 0.74985765, 0.69923893,
        0.67478045],
       [0.70700936, 0.76728803, 1.        , ..., 0.61287631, 0.54991866,
        0.56922461],
       ...,
       [0.74918614, 0.74985765, 0.61287631, ..., 1.        , 0.79791777,
        0.74379076],
       [0.69364512, 0.69923893, 0.54991866, ..., 0.79791777, 1.        ,
        0.7816415 ],
       [0.6932935 , 0.67478045, 0.56922461, ..., 0.74379076, 0.7816415 ,
        1.        ]])

In [142]:
averaged_df

,artist,vec_00,vec_000,vec_01,vec_02,vec_03,vec_04,vec_05,vec_06,vec_08,...,vec_흘러가버린,vec_흩날릴,vec_힘든,vec_힘들,vec_힘들게,vec_힘들어진,vec_힘은,vec_힘을,vec_ﬁxed,vec_𝘐𝘵
0,2 Chainz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21 Savage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21 Savage & Metro Boomin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2Pac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3LAU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5 Seconds of Summer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,50 Cent,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8Ball & MJG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,A Boogie Wit da Hoodie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,AC/DC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
def pull_genre(row):
    artist=row['artist'].replace(' ', '%20')
    CLIENT_ID = "d6629daf750946d3b1fcc4e7dc29c69a"
    CLIENT_SECRET = "90f081a4bb244f1a9e004530336ec2fc"

    grant_type = 'client_credentials'
    body_params = {'grant_type' : grant_type}

    url='https://accounts.spotify.com/api/token'
    response = requests.post(url, data=body_params, auth = (CLIENT_ID, CLIENT_SECRET)) 

    token_raw = json.loads(response.text)
    token = token_raw["access_token"]
    headers = {"Authorization": "Bearer {}".format(token)}
    try:
        r = requests.get(url="https://api.spotify.com/v1/search?q={}&type=artist".format(artist), headers=headers)
        ret = json.loads(r.text)
        return ret['artists']['items'][0]['genres']
    except:
        return []

In [145]:
averaged_df['Genre'] = averaged_df.apply(pull_genre,axis=1)



,artist,vec_00,vec_000,vec_01,vec_02,vec_03,vec_04,vec_05,vec_06,vec_08,...,vec_흩날릴,vec_힘든,vec_힘들,vec_힘들게,vec_힘들어진,vec_힘은,vec_힘을,vec_ﬁxed,vec_𝘐𝘵,Genre
0,2 Chainz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[atl hip hop, gangster rap, hip hop, pop, pop ..."
1,21 Savage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[atl hip hop, pop, pop rap, rap, southern hip ..."
2,21 Savage & Metro Boomin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[atl hip hop, pop, pop rap, rap, southern hip ..."
3,2Pac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[g funk, gangster rap, hip hop, rap, west coas..."
4,3LAU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[big room, dance pop, deep big room, edm, elec..."
5,5 Seconds of Summer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[boy band, dance pop, pop, post-teen pop]"
6,50 Cent,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[east coast hip hop, gangster rap, hip hop, po..."
7,8Ball & MJG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[crunk, deep southern trap, dirty south rap, g..."
8,A Boogie Wit da Hoodie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[melodic rap, pop rap, rap, trap]"
9,AC/DC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[album rock, australian rock, hard rock, rock]"


In [146]:
averaged_df

,artist,vec_00,vec_000,vec_01,vec_02,vec_03,vec_04,vec_05,vec_06,vec_08,...,vec_흩날릴,vec_힘든,vec_힘들,vec_힘들게,vec_힘들어진,vec_힘은,vec_힘을,vec_ﬁxed,vec_𝘐𝘵,Genre
0,2 Chainz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[atl hip hop, gangster rap, hip hop, pop, pop ..."
1,21 Savage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[atl hip hop, pop, pop rap, rap, southern hip ..."
2,21 Savage & Metro Boomin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[atl hip hop, pop, pop rap, rap, southern hip ..."
3,2Pac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[g funk, gangster rap, hip hop, rap, west coas..."
4,3LAU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[big room, dance pop, deep big room, edm, elec..."
5,5 Seconds of Summer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[boy band, dance pop, pop, post-teen pop]"
6,50 Cent,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[east coast hip hop, gangster rap, hip hop, po..."
7,8Ball & MJG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[crunk, deep southern trap, dirty south rap, g..."
8,A Boogie Wit da Hoodie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[melodic rap, pop rap, rap, trap]"
9,AC/DC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[album rock, australian rock, hard rock, rock]"


In [112]:
#averaged_df.to_csv("artist.csv", index=False)


,artist,vec_000,vec_02,vec_10,vec_100,vec_105,vec_108,vec_1099,vec_10s,vec_10th,...,vec_électron,vec_éloquent,vec_époque,vec_érigé,vec_ése,vec_était,vec_éthiopie,vec_être,vec_île,Genre
0,2Pac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[g funk, gangster rap, hip hop, rap, west coas..."
1,50 Cent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[east coast hip hop, gangster rap, hip hop, po..."
2,8Ball & MJG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[crunk, deep southern trap, dirty south rap, g..."
3,A Tribe Called Quest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[alternative hip hop, conscious hip hop, east ..."
4,Afrika Bambaataa & Soulsonic Force,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[east coast hip hop, old school hip hop]"
5,Arrested Development,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[atl hip hop, conscious hip hop, hip hop, rap]"
6,Asian Da Brat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[dfw rap, trap queen]"
7,Big Daddy Kane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[east coast hip hop, electro, gangster rap, ha..."
8,Big Pun,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[east coast hip hop, gangster rap, hardcore hi..."
9,Bone Thugs-N-Harmony,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[gangster rap, hip hop, pop rap, rap, southern..."
